## Chunking och Embedding

**Syfte:** Att ta alla våra rena `.txt`-filer, dela upp dem i hanterbara "chunks", omvandla dessa chunks till vektorer
(embeddings) och spara allt i en sökbar vektordatabas.

**Körs på:** Google Colab.

### Steg 1: Installationer
Vi installerar de bibliotek som behövs på Colab-miljön.
`sentence-transformers` är för vår lokala embedding-modell.
`langchain` har verktygen, och `chromadb` är vår databas.

In [ ]:
!pip install langchain-chroma
!pip install langchain-huggingface
!pip install langchain-text-splitters
!pip install sentence-transformers
!pip install tqdm
!pip install langchain-core

# %%
import os
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
import chromadb
import json

# Importer från LangChain
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_chroma import Chroma 
from langchain_huggingface import HuggingFaceEmbeddings

### Mountar min google drive för att komma åt min zip-fil med alla text-filer som ligger där

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Steg 2: Ladda upp och packa upp data
1. I fil-panelen till vänster i Colab, klicka på "Ladda upp".
2. Välj din lokalt zippade fil: `all_json_files.zip`.
3. Vänta tills uppladdningen är klar.
4. Kör cellen nedan för att packa upp filerna.

In [ ]:
# ----------------------------------------------------------------------
# OBS! Vi använder den absoluta sökvägen till filen på Google Drive.
# ----------------------------------------------------------------------

# Anpassad sökväg baserat på din bild
ZIP_FILE_PATH = Path("/content/drive/MyDrive/Green_Power_Sweden/all_json_files.zip")

# Detta är namnet på mappen där filerna ska packas upp (i Colabs temporära miljö, /content/)
UNZIP_DIR = "extracted_text"

# Vi använder .exists() på den fullständiga sökvägen till Drive
if ZIP_FILE_PATH.exists():
    # Hämtar bara filnamnet för utskriften
    ZIP_FILE_NAME = ZIP_FILE_PATH.name

    print(f"Hittade {ZIP_FILE_NAME} på Drive, packar upp till {UNZIP_DIR}...")

    # Rensa eventuell gammal data
    if Path(UNZIP_DIR).exists():
        print(f"Rensar gammal mapp: {UNZIP_DIR}")
        shutil.rmtree(UNZIP_DIR)

    # Använd f-strängar och !unzip
    # OBS: Vi måste skicka sökvägen som sträng till shell-kommandot (!unzip)
    !unzip -q "{ZIP_FILE_PATH}" -d {UNZIP_DIR}
    print(f"\n--- KLAR! ---")
    print(f"All text är uppackad till den temporära mappen '{UNZIP_DIR}'")
else:
    print(f"FEL: Kan inte hitta {ZIP_FILE_PATH}. Kontrollera att sökvägen är korrekt och att Drive är monterad.")

### Steg 3: Definiera sökvägar (på Colab)

Sätter upp var vi läser texten från och var vi
ska bygga vår nya databas.

In [ ]:
# Mappen där våra uppackade .txt-filer finns
TEXT_FILES_DIR = Path(UNZIP_DIR)

# Mappen där vi ska spara vår färdiga Chroma-databas
DB_PERSIST_DIR = Path("/content/drive/MyDrive/Green_Power_Sweden/green_power_sweden_db")

# Rensa eventuell gammal databas
if DB_PERSIST_DIR.exists():
    shutil.rmtree(DB_PERSIST_DIR)
DB_PERSIST_DIR.mkdir()

print(f"Läser textfiler från: {TEXT_FILES_DIR}")
print(f"Sparar vektordatabas till: {DB_PERSIST_DIR}")

### Steg 4: Ladda alla textdokument

Detta är ett kritiskt steg. Vi loopar igenom alla `.txt`-filer,
läser innehållet och skapar ett `Document`-objekt.

**Viktigast:** Vi sparar originalfilnamnet som `metadata`.
Vi rensar också bort `.txt`-suffixet för att få
det *riktiga* käll-filnamnet (t.ex. `dom_A_2023.pdf`).

In [ ]:
def load_all_texts(directory: Path) -> list[Document]:
    """Läser alla .json-filer och skapar LangChain-dokument med korrekt metadata."""
    documents = []
    
    print(f"Läser JSON-filer från {directory}...")
    all_files = list(directory.rglob("*.json")) # Vi letar efter JSON nu!
    
    for file_path in tqdm(all_files, desc="Laddar dokument"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Hämta vår fina metadata
            filename = data.get("filename", file_path.name)
            full_path = data.get("full_path", "Okänd sökväg") # T.ex. "domar/Kulturmiljö/fil.pdf"
            
            # Loopa igenom sidorna i JSON-filen
            for page in data.get("pages", []):
                page_text = page.get("text", "")
                page_num = page.get("page_number", 1)
                
                # Hoppa över tomma sidor
                if not page_text.strip():
                    continue

                # Skapa metadatan för just denna sida
                metadata = {
                    "source": filename,
                    "full_path": full_path, # Den viktiga sökvägen!
                    "page": page_num       # Sidnumret!
                }
                
                # Skapa dokumentet
                doc = Document(page_content=page_text, metadata=metadata)
                documents.append(doc)
            
        except Exception as e:
            print(f"Kunde inte läsa {file_path.name}: {e}")
            
    return documents

# Ladda alla dokument
all_documents = load_all_texts(TEXT_FILES_DIR)
print(f"\nKlar. Laddade {len(all_documents)} sidor/dokument.")
if all_documents:
    print(f"Exempel metadata: {all_documents[0].metadata}")

### Steg 5: Initiera Chunking-strategi

Vi använder `RecursiveCharacterTextSplitter`.
`chunk_size` = Max antal tecken per chunk.
`chunk_overlap` = Hur många tecken som ska överlappa
mellan chunks för att inte tappa kontexten.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400,
    separators=["\n\n", "\n", " ", ""] # Ordningen den försöker dela
)

print("Chunkar dokumenten...")
all_chunks = text_splitter.split_documents(all_documents)

print(f"Klar. Totalt antal dokument: {len(all_documents)}")
print(f"Totalt antal chunks skapade: {len(all_chunks)}")
print(f"Exempel på en chunks metadata: {all_chunks[0].metadata}")

### Steg 6: Initiera Embedding-modellen

Vi laddar ner `all-MiniLM-L12-v2` från HuggingFace.
Första gången du kör detta kommer Colab att ladda ner
modellen (tar ca 1-2 minuter).

Vi ställer in den att köras på `cuda` (GPU), vilket gör
processen extremt snabb.

In [ ]:
# model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'} # Använd GPU!
encode_kwargs = {'normalize_embeddings': False}

# print("Laddar embedding-modell (all-MiniLM-L12-v2)...")
print("Laddar embedding-modell (all-mpnet-base-v2)...")
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Modell laddad.")

### Steg 7: Skapa vektordatabasen

Nu tar vi alla våra chunks, vår embedding-modell och
talar om för ChromaDB att bygga databasen.

Detta är det tunga steget. Chroma kommer att:
1. Ta en chunk.
2. Skicka den till embedding-modellen -> får en vektor.
3. Spara vektorn och metadatan i databasen.
4. Upprepa för alla tusentals chunks.

Detta kan ta 10-30 minuter beroende på antal chunks.

In [ ]:
print(f"Bygger vektordatabas... Sparar till {DB_PERSIST_DIR}")

# Detta kommando gör allt: skapar och sparar databasen på disk
# Vi använder en tqdm-progress bar för att se hur det går
batch_size = 32 # Hur många chunks som ska processas åt gången
total_chunks = len(all_chunks)

# Vi måste göra det manuellt i en loop för att få en progress bar
db = Chroma(
    persist_directory=str(DB_PERSIST_DIR),
    embedding_function=embedding_model
)

# Loopa i batcher
for i in tqdm(range(0, total_chunks, batch_size), desc="Skapar embeddings"):
    batch = all_chunks[i:i + batch_size]
    db.add_documents(batch)

print("\n--- DATABAS BYGGD! ---")
print(f"Databasen är sparad i mappen {DB_PERSIST_DIR}.")
print(f"Totalt antal chunks i databasen: {db._collection.count()}")

### Steg 8: Zippa och ladda ner databasen

Nu zippar vi mappen som innehåller vår färdiga databas.
Därefter kan du högerklicka på filen `vector_db.zip`
i panelen till vänster och välja "Ladda ned".

In [ ]:
ZIP_DB_NAME = "/content/drive/MyDrive/Green_Power_Sweden/vector_db.zip"
print(f"Zippar databasmappen {DB_PERSIST_DIR} till {ZIP_DB_NAME}...")

!zip -r {ZIP_DB_NAME} {DB_PERSIST_DIR}

print(f"\nKlar! Din databas är redo.")
print(f"Ladda ner filen: {ZIP_DB_NAME} (från panelen till vänster)")